### 기본세팅

In [2]:
# 데이터 처리
import pandas as pd
import numpy as np

# 머신러닝
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
import lightgbm as lgb

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 기타
from datetime import datetime

In [4]:
customers_df = pd.read_csv("olist_customers_dataset.csv")
geo_df =  pd.read_csv("olist_geolocation_dataset.csv")
order_items_df =  pd.read_csv("olist_order_items_dataset.csv")
order_payments_df =  pd.read_csv("olist_order_payments_dataset.csv")
order_reviews_df =  pd.read_csv("olist_order_reviews_dataset.csv")
orders_df =  pd.read_csv("olist_orders_dataset.csv")
products_df =  pd.read_csv("olist_products_dataset.csv")
sellers_df =  pd.read_csv("olist_sellers_dataset.csv")
category_name_df =  pd.read_csv("product_category_name_translation.csv")

### 데이터 병합 & 시계열 전처리 

In [5]:
# 1. 주문(orders)과 고객(customers) 정보 결합
# 'customer_id'를 기준으로 병합
main_df = pd.merge(orders_df, customers_df, on="customer_id", how="left")

# 2. 주문 상품(order_items) 정보 추가
# 'order_id'를 기준으로 병합
main_df = pd.merge(main_df, order_items_df, on="order_id", how="left")

# 3. 상품 상세(products) 및 카테고리명(category_name) 정보 추가
# 'product_id' 기준으로 병합 후 영문 카테고리명으로 매핑.
main_df = pd.merge(main_df, products_df, on="product_id", how="left")
main_df = pd.merge(main_df, category_name_df, on="product_category_name", how="left")

# 4. 결제(order_payments) 및 리뷰(order_reviews) 정보 추가
main_df = pd.merge(main_df, order_payments_df, on="order_id", how="left")
main_df = pd.merge(main_df, order_reviews_df, on="order_id", how="left")

# 결과 확인
print(f"전체 병합된 데이터 행태: {main_df.shape}")
main_df.head()

전체 병합된 데이터 행태: (119143, 37)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017.10.2 10:56,2017.10.2 11:07,2017.10.4 19:55,2017.10.10 21:25,2017.10.18 0:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017.10.2 10:56,2017.10.2 11:07,2017.10.4 19:55,2017.10.10 21:25,2017.10.18 0:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017.10.2 10:56,2017.10.2 11:07,2017.10.4 19:55,2017.10.10 21:25,2017.10.18 0:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018.7.24 20:41,2018.7.26 3:24,2018.7.26 14:31,2018.8.7 15:27,2018.8.13 0:00,af07308b275d755c9edb36a90c618231,47813,...,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018.8.8 8:38,2018.8.8 8:55,2018.8.8 13:50,2018.8.17 18:06,2018.9.4 0:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,credit_card,3.0,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58


In [7]:
# 날짜 관련 컬럼 리스트 [cite: 3, 5, 6]
datetime_cols = [
    "order_purchase_timestamp", "order_approved_at", 
    "order_delivered_carrier_date", "order_delivered_customer_date", 
    "order_estimated_delivery_date", "review_creation_date", 
    "review_answer_timestamp"
]

# 날짜 데이터 타입 변환 (Object -> Datetime) [cite: 3, 6]
for col in datetime_cols:
    main_df[col] = pd.to_datetime(main_df[col])

# 분석용 월(Month) 파생변수 생성 
main_df['order_month'] = main_df['order_purchase_timestamp'].dt.to_period('M')

In [9]:
# 1. 시계열 데이터 변환 및 파생 변수 생성
# 배송 속도 및 지연 계산을 위한 필수 작업
date_cols = [
    "order_purchase_timestamp", "order_approved_at", 
    "order_delivered_carrier_date", "order_delivered_customer_date", 
    "order_estimated_delivery_date", "shipping_limit_date"
]

for col in date_cols:
    main_df[col] = pd.to_datetime(main_df[col])

# 배송 소요 시간(실제 배송일 - 구매일) 계산
main_df['delivery_actual_days'] = (main_df['order_delivered_customer_date'] - main_df['order_purchase_timestamp']).dt.days

# 배송 지연 여부 (실제 배송일 > 예정일)
main_df['is_delayed'] = main_df['order_delivered_customer_date'] > main_df['order_estimated_delivery_date']

# 2. 결측치 및 데이터 분포 확인 [cite: 3, 7]
# 2번 분석(제품 품질 vs 배송 문제)을 위해 리뷰 메시지 및 제품 정보 결측치 확인 [cite: 6, 8]
print(main_df[['review_comment_message', 'product_weight_g', 'order_delivered_customer_date']].isnull().sum())

review_comment_message           68898
product_weight_g                   853
order_delivered_customer_date     3421
dtype: int64


### EDA

#### 1번 _ 우수 셀러 정의 EDA

In [12]:
# 우수 셀러를 정의하기 위해 매출(Volume)과 고객 만족도(Review)의 분포를 확인

In [11]:
# 카테고리별 평균 리뷰 점수 및 매출 합계 [cite: 6, 8, 13]
category_analysis = main_df.groupby('product_category_name_english').agg({
    'review_score': 'mean',
    'payment_value': 'sum'
}).sort_values(by='payment_value', ascending=False)

# 주별(State) 고객 분포 확인 [cite: 10]
state_dist = main_df['customer_state'].value_counts()

#### 2번 _ 문제 감지 및 결제 데이터 구조 확인 EDA

##### 지연 징후 포착과 결제 성공 여부를 판단하기 위한 사전 탐색

In [ ]:
# 결제 데이터 구조 확인 (성공 위주 여부)

# order_patments는 승인된 결제 위주. 할부 정보는 있으나 거절 로그는 확인이 어려움.
# 고로 승인 시간 부재를 결제 단계 문제로 간주 

In [13]:
# 결제 승인 지연 확인 (구매 시각 vs 승인 시각)
main_df['approval_delta'] = (main_df['order_approved_at'] - main_df['order_purchase_timestamp']).dt.total_seconds() / 3600

# 승인되지 않은 주문(결제 오류 가능성) 비율 확인 [cite: 3]
unapproved_rate = main_df['order_approved_at'].isnull().mean()
print(f"결제 미승인 주문 비율: {unapproved_rate:.2%}")

결제 미승인 주문 비율: 0.15%


##### 지역별 물류 병목 및 지연 징후 

In [ ]:
# 특정 지역 (customer_state)에서 배송 지연이 집중되는지 확인 
# 물류 병목 현상 파악 

In [14]:
# 지역별 평균 배송 지연율 및 평균 배송 소요 기간
state_delay_analysis = main_df.groupby('customer_state').agg({
    'is_delayed': 'mean',
    'delivery_actual_days': 'mean'
}).sort_values(by='is_delayed', ascending=False)

# 지연 징후 주문 추출 (Golden Time: shipping_limit_date + 48시간 초과 건) [cite: 1]
from datetime import timedelta
current_logic_time = main_df['shipping_limit_date'] + timedelta(hours=48)
potential_delay_orders = main_df[
    (main_df['order_status'].isin(['processing', 'order_approved'])) & 
    (pd.Timestamp.now() > current_logic_time) # 현재 시점이 골든타임을 지났을 때
]

### 1번 _ 우수 셀러 행동 패턴 추출 

In [ ]:
# 기준 매출 상위 10%면서, 평균 리뷰 4.5이상 셀러 excellent 그룹 분리 
# 비교 지표 : 주문처리 속도, 상품 정보 상세도 (이미지 수, 설명 길이), 카테고리 집중도 

In [17]:
# 셀러별 행동 데이터 집계
seller_behavior = main_df.groupby('seller_id').agg({
    'order_id': 'nunique',
    'payment_value': 'sum',
    'review_score': 'mean',
    'product_photos_qty': 'mean',
    'product_description_lenght': 'mean',
    'approval_delta': 'mean',
    'delivery_actual_days': 'mean'
}).reset_index()

# 그룹 분리 (Excellent vs Others)
revenue_threshold = seller_behavior['payment_value'].quantile(0.9)
seller_behavior['group'] = 'Others'
seller_behavior.loc[(seller_behavior['payment_value'] >= revenue_threshold) & 
                    (seller_behavior['review_score'] >= 4.5), 'group'] = 'Excellent'

# 숫자형 컬럼만 평균 계산하도록 numeric_only=True 추가
comparison = seller_behavior.groupby('group').mean(numeric_only=True)

# 결과 확인
print("우수 셀러 vs 일반 셀러 패턴 비교:")
print(comparison[['product_photos_qty', 'product_description_lenght', 'delivery_actual_days']])

우수 셀러 vs 일반 셀러 패턴 비교:
           product_photos_qty  product_description_lenght  \
group                                                       
Excellent            2.149649                 1100.724892   
Others               2.248439                  857.634254   

           delivery_actual_days  
group                            
Excellent             11.404274  
Others                11.712099  


### 2번 _ 리뷰 전 문제 감지 로직 구체화 

In [ ]:
# 지연 징후 포착 -> 고가치 고객 분류 
# 물류 데이터를 기반으로 징후를 먼저 포착하는 로직 

# 골든 타임 : shipping limit date 기준 +48시간 경과 시점을 임계치
# 타겟팅 : 첫 구매 고객이나 고액 결제 고객에게 우선 순위 부여 

In [18]:
from datetime import timedelta

# 1. 지연 위험 주문 추출 (Golden Time: 48시간 기준)
# 데이터셋의 마지막 배송 제한일을 기준으로 시뮬레이션 [cite: 1]
max_limit_date = main_df['shipping_limit_date'].max()
main_df['is_risk'] = (main_df['shipping_limit_date'] + timedelta(hours=48) < max_limit_date) & \
                     (main_df['order_status'].isin(['processing', 'approved']))

# 2. 고가치 및 첫 구매 고객 식별 (우선 순위 설정용)
# 고객별 주문 횟수 계산 [cite: 3, 10]
customer_counts = main_df.groupby('customer_unique_id')['order_id'].transform('nunique')
main_df['is_first_purchase'] = customer_counts == 1
# 결제 금액 상위 20%를 고가치 고객으로 정의 [cite: 13]
main_df['is_high_value'] = main_df['payment_value'] > main_df['payment_value'].quantile(0.8)

# 3. 자동 액션 타겟팅 리스트
# 지연 위험이 있으면서 고가치 고객이거나 첫 구매인 경우 [cite: 1, 13]
priority_action_list = main_df[main_df['is_risk'] & (main_df['is_first_purchase'] | main_df['is_high_value'])]

print(f"즉시 대응 필요 타겟 수: {len(priority_action_list)}건")

즉시 대응 필요 타겟 수: 362건


In [ ]:
# 특정 지역에서 발생하는 지연을 파악해 셀러의 문제인지 물류의 문제인지 구분 

In [19]:
# 주(State)별 지연율 분석 [cite: 10]
state_bottleneck = main_df.groupby('customer_state').agg({
    'is_delayed': 'mean',
    'order_id': 'count'
}).rename(columns={'is_delayed': 'delay_rate', 'order_id': 'order_count'})

# 지연율이 높은 위험 지역 TOP 5 [cite: 10, 11]
top_bottlenecks = state_bottleneck.sort_values(by='delay_rate', ascending=False).head(5)

print("물류 병목 의심 지역:")
print(top_bottlenecks)

물류 병목 의심 지역:
                delay_rate  order_count
customer_state                         
AL                0.234914          464
MA                0.193925          856
PI                0.152778          576
SE                0.151365          403
CE                0.144409         1565


### 1번 _ 우수 셀러 카테고리 분포 및 행동 패턴

In [ ]:
# 특정 카테고리에 몰려있는지, 카테고리와 상관없이 공통된 행동 (사진수, 설명 길이)보이는지 증명 

In [20]:
# 우수 셀러가 속한 주요 카테고리 분석
excellent_seller_ids = seller_behavior[seller_behavior['group'] == 'Excellent']['seller_id']
excellent_main_df = main_df[main_df['seller_id'].isin(excellent_seller_ids)]

# 카테고리별 우수 셀러 비중 확인
category_excellence = excellent_main_df.groupby('product_category_name_english').agg({
    'order_id': 'nunique',
    'review_score': 'mean'
}).sort_values(by='order_id', ascending=False).head(10)

print("--- [1번] 우수 셀러 주요 활동 카테고리 TOP 10 ---")
print(category_excellence)

--- [1번] 우수 셀러 주요 활동 카테고리 TOP 10 ---
                               order_id  review_score
product_category_name_english                        
cool_stuff                          214      4.518182
perfumery                           104      4.577586
auto                                 87      4.649485
toys                                 84      4.666667
housewares                           74      4.457831
watches_gifts                        67      4.611111
home_appliances_2                    41      4.642857
health_beauty                        38      4.674419
pet_shop                             38      4.666667
musical_instruments                  31      4.568182


In [ ]:
# 우수 셀러는 일반 셀러보다 평균적으로 더 많은 상품 사진 (product_photos_qty)를 등록 
# 상품 설명 길이와 리뷰 점수간 상관관계 (제시)

### 2번 _ 지연 위험군의 리뷰 점수 하락 폭 분석 

In [ ]:
# 선제대응 (쿠폰/알림)이 왜 필요한지 데이터로 증명하기 위해,
# 지연된 주문이 리뷰 점수에 미치는 치명적 영향 수치화 

In [23]:
# 배송 지연 여부에 따른 리뷰 점수 차이 분석
delay_impact = main_df.groupby('is_delayed').agg({
    'review_score': 'mean',
    'payment_value': 'mean'
}).reset_index()

# 지연된 주문 중 '배송 문제' vs '제품 문제' 구분 (리뷰 텍스트 유무 활용)
# 리뷰 메시지가 있는 경우 배송 지연 시 불만족도가 더 구체적으로 표출됨
main_df['has_review_message'] = main_df['review_comment_message'].notnull()
message_impact = main_df.groupby(['is_delayed', 'has_review_message'])['review_score'].mean().unstack()

print("--- [2번] 배송 지연이 리뷰 점수에 미치는 영향 ---")
print(delay_impact)
print("\n리뷰 메시지 유무에 따른 점수 차이:")
print(message_impact)

--- [2번] 배송 지연이 리뷰 점수에 미치는 영향 ---
   is_delayed  review_score  payment_value
0       False      4.134724     172.228512
1        True      2.546542     178.886992

리뷰 메시지 유무에 따른 점수 차이:
has_review_message     False     True 
is_delayed                            
False               4.437710  3.706519
True                3.112506  2.101391


In [ ]:
# is_delayed 가 true 일 때, 리뷰점수 급격히 하락 
# shipping_limit_date 초과시점  = 골든타임 
# 고가치 고객의 이탈은 일반 고객보다 매출 타격이 큼 = 선제적 보상이 경제적 